In [1]:

import dataretrieval.nwis as nwis
import geopandas as gpd
from shapely.geometry import Point, box, Polygon, MultiPolygon
import requests
import pandas as pd
import datetime as dt
import earthaccess



In [7]:

# set parameters for search
param_codes = ['63680'] # turbidity
param_codes_str = ','.join(param_codes) 
state_code = '06'
start_date = '2023-01-01'
end_date = '2023-12-31'


def get_param_sites(param_codes_str, state_code):
""" retrieve sites with available data for parameter codes """

    url = f"https://waterservices.usgs.gov/nwis/site/?format=rdb&stateCd={state_code}&parameterCd={param_codes_str}&startDT={start_date}&endDT={end_date}"
    response = requests.get(url)
    
    if response.status_code == 200:
        lines = response.text.splitlines()
        data_lines = [line for line in lines if not line.startswith('#') and line.strip()]
        
        if data_lines:
            headers = data_lines[0].split('\t')  # First line contains headers
            data_lines = [line.split('\t') for line in data_lines[2:]]  # Process the remaining data
            df = pd.DataFrame(data_lines, columns=headers)
            df_filter = df[['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va']]
            return df_filter
    return pd.Dataframe()


# function calls, prints
site_data = get_param_sites(param_codes_str, state_code)
#df_sites = pd.DataFrame(site_data)
site_head_test = site_data.head()
print(site_data.head())
#site_data.to_csv('chla_sites_california.csv', index=False)



    site_no                                         station_nm   dec_lat_va  \
0  09429500           COLORADO RIVER BELOW IMPERIAL DAM, AZ-CA  32.86758333   
1  09522990  ALL-AMERICAN CANAL HEADWRKS AT IMPERIAL DAM, C...  32.88494444   
2  10336610           UPPER TRUCKEE RV AT SOUTH LAKE TAHOE, CA   38.9224078   
3  10336645                          GENERAL C NR MEEKS BAY CA  39.05185197   
4  10336660                       BLACKWOOD C NR TAHOE CITY CA  39.10740708   

    dec_long_va  
0   -114.472111  
1   -114.468111  
2  -119.9915706  
3  -120.1185209  
4  -120.1621352  


In [3]:

# setup for granule search
doi = '10.5067/EMIT/EMITL2ARFL.001'
cmrurl = 'https://cmr.earthdata.nasa.gov/search/'
doisearch = cmrurl + 'collections.json?doi=' + doi
concept_id = requests.get(doisearch).json()['feed']['entry'][0]['id']
start_date = dt.datetime(2021, 9, 3)
end_date = dt.datetime(2023, 9, 3)  
dt_format = '%Y-%m-%dT%H:%M:%SZ'
temporal_str = start_date.strftime(dt_format) + ',' + end_date.strftime(dt_format)


def get_site_granules(site_no, site_lat, site_lon):
""" retrieve granules based on site coordinates """

    page_num = 1
    page_size = 2000
    granule_arr = []
    
    while True:
        cmr_param = {
            "collection_concept_id": concept_id,
            "page_size": page_size,
            "page_num": page_num,
            "temporal": temporal_str,
            "point": f"{site_lon},{site_lat}"  # Longitude first, then Latitude
        }
    
        granulesearch = cmrurl + 'granules.json'
        response = requests.post(granulesearch, data=cmr_param)
        granules = response.json()['feed']['entry']
    
        if granules:
            for g in granules:
                granule_urls = [x['href'] for x in g['links'] if 'https' in x['href'] and '.nc' in x['href'] and '.dmrpp' not in x['href']]
                granule_datetime = g.get('time_start', 'N/A')
    
                if 'polygons' in g:
                    poly_coords = g['polygons'][0][0].split(" ")
                    poly_coords = [[float(poly_coords[i+1]), float(poly_coords[i])] for i in range(0, len(poly_coords), 2)]
                    polygon = Polygon(poly_coords)
                    granule_bbox = polygon.bounds  # Bounding box (min_lon, min_lat, max_lon, max_lat)
                else:
                    granule_bbox = 'N/A'
                
                granule_arr.append({
                    "site_no": site_no,
                    "site_lat": site_lat,
                    "site_lon": site_lon,
                    "granule_urls": granule_urls,
                    "granule_bbox": granule_bbox,
                    "datetime": granule_datetime
                })
                
            page_num += 1
        else:
            break
    return granule_arr


def get_all_site_granules(site_data):
""" combine data across sites """

    all_granules = []
    for i, row in site_data.iterrows():
        site_no = row['site_no']
        site_lat = row['dec_lat_va']
        site_lon = row['dec_long_va']
        
        print(f"Retrieving granules for site: {site_no}")    
        site_granules = get_site_granules(site_no, site_lat, site_lon)
        all_granules.extend(site_granules)
    
    return all_granules


# function calls, prints
all_site_granules = get_all_site_granules(site_head_test)
df_granules = pd.DataFrame(all_site_granules)
granule_head_test = df_granules.head()
#print(all_site_granules)
print(df_granules.head())




2021-09-03T00:00:00Z,2023-09-03T00:00:00Z
Retrieving granules for site: 09429500
Retrieving granules for site: 09522990
Retrieving granules for site: 10336610
Retrieving granules for site: 10336645
Retrieving granules for site: 10336660
    site_no     site_lat     site_lon  \
0  09429500  32.86758333  -114.472111   
1  09429500  32.86758333  -114.472111   
2  09429500  32.86758333  -114.472111   
3  09429500  32.86758333  -114.472111   
4  09429500  32.86758333  -114.472111   

                                        granule_urls  \
0  [https://data.lpdaac.earthdatacloud.nasa.gov/l...   
1  [https://data.lpdaac.earthdatacloud.nasa.gov/l...   
2  [https://data.lpdaac.earthdatacloud.nasa.gov/l...   
3  [https://data.lpdaac.earthdatacloud.nasa.gov/l...   
4  [https://data.lpdaac.earthdatacloud.nasa.gov/l...   

                                        granule_bbox                  datetime  
0  (-114.6581268, 32.2750206, -113.4960022, 33.42...  2022-08-13T23:26:05.000Z  
1  (-115.229248, 

In [6]:

def get_granule_results(site_no, param_cd, granule_time, time_window=5440):
""" retrieve results for site and granule time """
    results = []

    start_time = (granule_time - pd.Timedelta(minutes=time_window)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = (granule_time + pd.Timedelta(minutes=time_window)).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    url = f"https://waterservices.usgs.gov/nwis/iv/?format=json&sites={site_no}&parameterCd={param_cd}&startDT={start_time}&endDT={end_time}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if 'value' in data and 'timeSeries' in data['value']:
            for ts in data['value']['timeSeries']:
                for value in ts['values'][0]['value']:
                    results.append({
                        'chla_measurement': value['value'],
                        'chla_unit': ts['variable']['unit']['unitCode'],
                        'chla_time': value['dateTime']
                    })
    else:
        print(f"Failed to fetch Chla data for site {site_no}: {response.status_code}")
    
    return results


# match each granule to chla results
def match_granules_chla(df_granules, param_codes):
""" match results with granules dataframe """
    
    results = []

    for index, row in df_granules.iterrows():
        site_no = row['site_no']
        granule_time = pd.to_datetime(row['datetime']) 

        for param_cd in param_codes:
            chla_data = get_granule_results(site_no, param_cd, granule_time)
            if chla_data:
                for chla in chla_data:
                    results.append({
                        'site_no': site_no,
                        'granule_time': granule_time,
                        'granule_bbox': row['granule_bbox'],
                        'granule_urls': row['granule_urls'],
                        'chla_measurement': chla['chla_measurement'],
                        'chla_unit': chla['chla_unit'],
                        'chla_time': chla['chla_time']
                    })

    return pd.DataFrame(results)


# function calls, prints
df_granules_chla = match_granules_chla(granule_head_test, param_codes)
print(df_granules_chla.head())



Empty DataFrame
Columns: []
Index: []
